In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
print(f"Torch Version: {torch.__version__}")

import transformers
print(f"transformers (Adapter) Version: {transformers.__version__}")

Torch Version: 1.8.1
transformers (Adapter) Version: 2.0.1


In [3]:
from transformers import RobertaTokenizer
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

In [4]:
from ner_dataset import get_trainset_data_loader

all_tags, trainset, trainloader = get_trainset_data_loader(tokenizer, BATCH_SIZE=16)

In [6]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=1
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

name = model.load_adapter("./save_adapters/ALL_tag_0730")
model.add_tagging_head(
        name,
        num_labels=len(trainset.label_map.keys()), overwrite_ok=True
      )
model.train_adapter(name)


no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
                {
                    "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                    "weight_decay": 1e-5,
                },
                {
                    "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                    "weight_decay": 0.0,
                },
            ]
optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=1e-4)


In [7]:
def get_optimizer(model):
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
                    {
                        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                        "weight_decay": 1e-5,
                    },
                    {
                        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                        "weight_decay": 0.0,
                    },
                ]
    optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=1e-4)
    return optimizer

In [8]:
all_tags = ['Float','TemporalUnit','I-gpe','CountryCode','CurrencyCode','Timezone','CryptoCurrencyCode','Month','Party','B-tim','I-art','Time','B-per','B-gpe','B-geo','O','Location','Event','I-nat','Race','B-org','I-geo','I-tim','I-eve','SpecialTerm','B-art','US_States','B-eve','I-org','B-nat','Object','I-per','Integer']

In [1]:
from telegram_notifier import send_message as telegram_bot_sendtext

In [10]:
train_id = 1
device = torch.device(f"cuda:{train_id}" if torch.cuda.is_available() else "cpu")

#model = model.to(device)

In [11]:
tag = "Party"
try:
    model.add_adapter(tag)
    model.add_tagging_head(
        tag,
        num_labels=1
      )
except: pass
model.train_adapter(tag)
model = model.to(device)
optimizer = get_optimizer(model)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [13]:
for i, data in enumerate(trainloader):

    tokens_tensors, segments_tensors, \
    masks_tensors, labels = [t.to(device) for t in data]

    outputs = model(input_ids = tokens_tensors,
        attention_mask=masks_tensors,
        token_type_ids=segments_tensors)



    logits = outputs[0]

    active_logits = logits.view(-1, logits.shape[-1])[masks_tensors.view(-1) == 1]

    active_labels = labels.view(-1, logits.shape[-1])[masks_tensors.view(-1)== 1]

    loss_fct = torch.nn.BCEWithLogitsLoss()

    loss = loss_fct(active_logits, active_labels)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i % 1 == 0:
        print(f"loss: {loss}")

IndexError: The shape of the mask [672] at index 0 does not match the shape of the indexed tensor [22176, 1] at index 0

In [14]:
for i, data in enumerate(trainloader):

    tokens_tensors, segments_tensors, \
    masks_tensors, labels = [t.to(device) for t in data]

    outputs = model(input_ids = tokens_tensors,
        attention_mask=masks_tensors,
        token_type_ids=segments_tensors)


    logits = outputs[0]

    current_label = labels.view(-1, labels.shape[-1])[:, trainset.label_map[tag]]
    current_label = current_label.view(-1)
    active_labels = current_label[masks_tensors.view(-1)== 1]
    actual = current_label[masks_tensors.view(-1)== 1].float().view(-1,1)
    
    active_logits = logits.view(-1, logits.shape[-1])[masks_tensors.view(-1) == 1]
    active_logits = active_logits + 0.5
    
    """active_labels = labels.view(-1, logits.shape[-1])[masks_tensors.view(-1)== 1]
    actual = active_labels
    

    actual = torch.ones(active_logits.shape, device = device)

    actual[:, 0] = (active_labels == 0).long()
    actual[:, 1] = (active_labels == 1).long()"""


    loss_fct = torch.nn.BCEWithLogitsLoss()

    loss = loss_fct(active_logits, actual)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i % 1 == 0:
        print(f"\tLoss: {loss}")

	Loss: 0.0700686052441597
	Loss: 0.061259426176548004
	Loss: 0.07845277339220047
	Loss: 0.07074683159589767
	Loss: 0.03633202612400055
	Loss: 0.07647769898176193
	Loss: 0.06860431283712387
	Loss: 0.12048780918121338
	Loss: 0.046974729746580124
	Loss: 0.11914849281311035
	Loss: 0.07654492557048798
	Loss: 0.06592464447021484
	Loss: 0.04424158111214638
	Loss: 0.10774428397417068
	Loss: 0.11361988633871078


KeyboardInterrupt: 

In [ ]:
for i, data in enumerate(trainloader):

    tokens_tensors, segments_tensors, \
    masks_tensors, labels = [t.to(device) for t in data]

    outputs = model(input_ids = tokens_tensors,
        attention_mask=masks_tensors,
        token_type_ids=segments_tensors)


    logits = outputs[0]

    current_label = labels.view(-1, labels.shape[-1])[:, trainset.label_map[tag]]
    current_label = current_label.view(-1)
    active_labels = current_label[masks_tensors.view(-1)== 1]
    actual = current_label[masks_tensors.view(-1)== 1].float().view(-1,1)
    
    active_logits = logits.view(-1, logits.shape[-1])[masks_tensors.view(-1) == 1]
    active_logits = active_logits + 0.5
    
    """active_labels = labels.view(-1, logits.shape[-1])[masks_tensors.view(-1)== 1]
    actual = active_labels
    

    actual = torch.ones(active_logits.shape, device = device)

    actual[:, 0] = (active_labels == 0).long()
    actual[:, 1] = (active_labels == 1).long()"""


    loss_fct = torch.nn.BCEWithLogitsLoss()

    loss = loss_fct(active_logits, actual)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i % 1 == 0:
        print(f"\tLoss: {loss}")

	Loss: 0.7552669048309326
	Loss: 0.751865804195404
	Loss: 0.7435103058815002
	Loss: 0.747766375541687
	Loss: 0.7505049109458923
	Loss: 0.7498570680618286
	Loss: 0.748807430267334
	Loss: 0.7292674779891968
	Loss: 0.7384851574897766
	Loss: 0.7338241934776306
	Loss: 0.7388399839401245
	Loss: 0.7284795045852661
	Loss: 0.73432856798172
	Loss: 0.7380419969558716
	Loss: 0.730492115020752
	Loss: 0.7304741144180298
	Loss: 0.7195546627044678
	Loss: 0.7301626205444336
	Loss: 0.7192289233207703
	Loss: 0.72626793384552
	Loss: 0.7132450342178345
	Loss: 0.7216485142707825
	Loss: 0.7014139294624329
	Loss: 0.7065895795822144
	Loss: 0.7152504324913025
	Loss: 0.6977663636207581
	Loss: 0.7163938879966736
	Loss: 0.6975769400596619
	Loss: 0.7093460559844971
	Loss: 0.7066753506660461
	Loss: 0.7021980285644531
	Loss: 0.7039337754249573
	Loss: 0.7046786546707153
	Loss: 0.6993280649185181
	Loss: 0.7070374488830566
	Loss: 0.6902365684509277
	Loss: 0.6933871507644653
	Loss: 0.6774394512176514
	Loss: 0.69632291793

	Loss: 0.3637509346008301
	Loss: 0.3707278370857239
	Loss: 0.3502502739429474
	Loss: 0.25905925035476685
	Loss: 0.4013005495071411
	Loss: 0.282198429107666
	Loss: 0.2457723468542099
	Loss: 0.44087985157966614
	Loss: 0.23479662835597992
	Loss: 0.2768978476524353
	Loss: 0.3170405924320221
	Loss: 0.3553886115550995
	Loss: 0.26999086141586304
	Loss: 0.24485526978969574
	Loss: 0.30054807662963867
	Loss: 0.2492661029100418
	Loss: 0.27368542551994324
	Loss: 0.2139972746372223
	Loss: 0.33631524443626404
	Loss: 0.3368043303489685
	Loss: 0.4384312033653259
	Loss: 0.3300243318080902
	Loss: 0.338998019695282
	Loss: 0.2853093445301056
	Loss: 0.28079095482826233
	Loss: 0.3004574477672577
	Loss: 0.24238048493862152
	Loss: 0.3147948682308197
	Loss: 0.3357212543487549
	Loss: 0.27010372281074524
	Loss: 0.2908060848712921
	Loss: 0.28202518820762634
	Loss: 0.34455716609954834
	Loss: 0.29230257868766785
	Loss: 0.24870355427265167
	Loss: 0.2321203500032425
	Loss: 0.13117027282714844
	Loss: 0.353260010480880

	Loss: 0.18487517535686493
	Loss: 0.22701029479503632
	Loss: 0.2900080382823944
	Loss: 0.28217944502830505
	Loss: 0.21440187096595764
	Loss: 0.23066593706607819
	Loss: 0.22506050765514374
	Loss: 0.19444842636585236
	Loss: 0.31348997354507446
	Loss: 0.2251330465078354
	Loss: 0.18478746712207794
	Loss: 0.22262272238731384
	Loss: 0.19085875153541565
	Loss: 0.25121545791625977
	Loss: 0.2499750256538391
	Loss: 0.2418396770954132
	Loss: 0.19061557948589325
	Loss: 0.19746971130371094
	Loss: 0.28817883133888245
	Loss: 0.2611553370952606
	Loss: 0.21424834430217743
	Loss: 0.20216238498687744
	Loss: 0.24244634807109833
	Loss: 0.22592727839946747
	Loss: 0.19864705204963684
	Loss: 0.2198340892791748
	Loss: 0.2379891723394394
	Loss: 0.15783055126667023
	Loss: 0.0732632502913475
	Loss: 0.134937584400177
	Loss: 0.22456806898117065
	Loss: 0.20581446588039398
	Loss: 0.16499082744121552
	Loss: 0.25325557589530945
	Loss: 0.2021835893392563
	Loss: 0.20211991667747498
	Loss: 0.2941252291202545
	Loss: 0.2429

In [51]:
for index, tag in enumerate(all_tags):
    if index % 2 == train_id:
        print(f"\nSkip {tag}.\n")
        continue
    tag = "Party"
    try:
        model.add_adapter(tag)
        model.add_tagging_head(
            tag,
            num_labels=1
          )
    except: pass
    model.train_adapter(tag)
    model = model.to(device)
    for epoch in range(2):
        print(f"\n{tag}: epoch {epoch}")
        for i, data in enumerate(trainloader):

            tokens_tensors, segments_tensors, \
            masks_tensors, labels = [t.to(device) for t in data]

            outputs = model(input_ids = tokens_tensors,
                attention_mask=masks_tensors,
                token_type_ids=segments_tensors)


            logits = outputs[0]

            current_label = labels.view(-1, labels.shape[-1])[:, trainset.label_map[tag]]
            current_label = current_label.view(-1)

            active_logits = logits.view(-1, logits.shape[-1])[masks_tensors.view(-1) == 1]
            active_labels = current_label[masks_tensors.view(-1)== 1]

            actual = current_label[masks_tensors.view(-1)== 1].float().view(-1,1)
            """

            actual = torch.ones(active_logits.shape, device = device)

            actual[:, 0] = (active_labels == 0).long()
            actual[:, 1] = (active_labels == 1).long()"""


            loss_fct = torch.nn.BCEWithLogitsLoss()

            loss = loss_fct(active_logits, actual)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if i % 1 == 0:
                print(f"\tLoss: {loss}")
        telegram_bot_sendtext(f"\n{tag}: epoch {epoch}, loss = {loss}")
    model.save_adapter(f"./save_adapters/{tag}_0730", model.active_adapters[0])
    model.save_head(f"./save_heads/{tag}_0730", model.active_head)


Party: epoch 0
	Loss: 0.6123307347297668
	Loss: 0.61722332239151
	Loss: 0.6266659498214722
	Loss: 0.6249571442604065
	Loss: 0.6243467330932617
	Loss: 0.6254064440727234
	Loss: 0.6210866570472717
	Loss: 0.6185749173164368
	Loss: 0.6312183141708374
	Loss: 0.6246612071990967
	Loss: 0.6151171326637268
	Loss: 0.6085354685783386
	Loss: 0.6208930015563965
	Loss: 0.623403012752533
	Loss: 0.6351619362831116
	Loss: 0.6257862448692322
	Loss: 0.6235737800598145
	Loss: 0.6237320303916931
	Loss: 0.6235618591308594
	Loss: 0.6198935508728027
	Loss: 0.6148433685302734
	Loss: 0.6220027804374695
	Loss: 0.6236675381660461
	Loss: 0.6199929714202881
	Loss: 0.6227532625198364
	Loss: 0.6157484650611877
	Loss: 0.654728889465332
	Loss: 0.6165981292724609
	Loss: 0.6331440210342407
	Loss: 0.6283099055290222
	Loss: 0.6230689883232117
	Loss: 0.6227986812591553
	Loss: 0.6219102740287781
	Loss: 0.6328703761100769
	Loss: 0.6271761655807495
	Loss: 0.610309898853302
	Loss: 0.6161659359931946
	Loss: 0.6296997666358948
	

	Loss: 0.6321003437042236
	Loss: 0.6169648766517639
	Loss: 0.6229113340377808
	Loss: 0.6390840411186218
	Loss: 0.620663046836853
	Loss: 0.6210998296737671
	Loss: 0.6229880452156067
	Loss: 0.6223722696304321
	Loss: 0.6199183464050293
	Loss: 0.6160904169082642
	Loss: 0.6230624318122864
	Loss: 0.6173529028892517
	Loss: 0.6156640648841858
	Loss: 0.6127409338951111
	Loss: 0.6289763450622559
	Loss: 0.6270835399627686
	Loss: 0.6421941518783569
	Loss: 0.6246063709259033
	Loss: 0.6323327422142029
	Loss: 0.624030590057373
	Loss: 0.618242084980011
	Loss: 0.6207431554794312
	Loss: 0.6194913983345032
	Loss: 0.6216254234313965
	Loss: 0.6309607625007629
	Loss: 0.6116876006126404
	Loss: 0.6280983686447144
	Loss: 0.621906578540802
	Loss: 0.6228669285774231
	Loss: 0.6205018758773804
	Loss: 0.6191455721855164
	Loss: 0.609718382358551
	Loss: 0.6094355583190918
	Loss: 0.6334944367408752
	Loss: 0.6165165305137634
	Loss: 0.6248624324798584
	Loss: 0.63666170835495
	Loss: 0.6300413608551025
	Loss: 0.6162485480

	Loss: 0.6299533843994141
	Loss: 0.617529034614563
	Loss: 0.6245426535606384
	Loss: 0.6318857669830322
	Loss: 0.636189341545105
	Loss: 0.6163477301597595
	Loss: 0.6271756887435913
	Loss: 0.6311674118041992
	Loss: 0.623897135257721
	Loss: 0.6285223364830017
	Loss: 0.6245979070663452
	Loss: 0.6278209090232849
	Loss: 0.6261910200119019
	Loss: 0.6218616366386414
	Loss: 0.623475193977356
	Loss: 0.6155397891998291
	Loss: 0.6126983165740967
	Loss: 0.6235607862472534
	Loss: 0.6151019930839539
	Loss: 0.6293394565582275
	Loss: 0.6280133724212646
	Loss: 0.6137151122093201
	Loss: 0.619037926197052
	Loss: 0.6166980862617493
	Loss: 0.6347509622573853
	Loss: 0.6375947594642639
	Loss: 0.6168099641799927
	Loss: 0.6207336783409119
	Loss: 0.6136570572853088
	Loss: 0.6232959628105164
	Loss: 0.6366313099861145
	Loss: 0.6121021509170532
	Loss: 0.6383124589920044
	Loss: 0.6257652044296265
	Loss: 0.630351722240448
	Loss: 0.6268367171287537
	Loss: 0.6361743807792664
	Loss: 0.6286131143569946
	Loss: 0.631359159

	Loss: 0.6151708960533142
	Loss: 0.6374208331108093
	Loss: 0.6085429787635803
	Loss: 0.6188129186630249
	Loss: 0.6169146299362183
	Loss: 0.6391388773918152
	Loss: 0.615227997303009
	Loss: 0.6243712902069092
	Loss: 0.6257045269012451
	Loss: 0.6236923336982727
	Loss: 0.635103166103363
	Loss: 0.6386023759841919
	Loss: 0.6052246689796448
	Loss: 0.6267717480659485
	Loss: 0.6226387023925781
	Loss: 0.623080313205719
	Loss: 0.6272146105766296
	Loss: 0.6232014894485474
	Loss: 0.6273452043533325
	Loss: 0.623246967792511
	Loss: 0.6280873417854309
	Loss: 0.6220138072967529
	Loss: 0.6312389373779297
	Loss: 0.6390282511711121
	Loss: 0.6152775883674622
	Loss: 0.6192253828048706
	Loss: 0.6127772927284241
	Loss: 0.6324679851531982
	Loss: 0.6248519420623779
	Loss: 0.6176460981369019
	Loss: 0.6324719786643982
	Loss: 0.6273760795593262
	Loss: 0.6331645846366882
	Loss: 0.6379944682121277
	Loss: 0.6170519590377808
	Loss: 0.6323664784431458
	Loss: 0.6251636147499084
	Loss: 0.6267881393432617
	Loss: 0.6267179

	Loss: 0.6324782967567444
	Loss: 0.6236677169799805
	Loss: 0.6308465600013733
	Loss: 0.6156812310218811
	Loss: 0.6233917474746704
	Loss: 0.6223833560943604
	Loss: 0.6199033260345459
	Loss: 0.6188119649887085
	Loss: 0.6336750984191895
	Loss: 0.622198760509491
	Loss: 0.6326879262924194
	Loss: 0.6320456266403198
	Loss: 0.6264495253562927
	Loss: 0.6265836358070374
	Loss: 0.6152316927909851
	Loss: 0.6239086389541626
	Loss: 0.6182982325553894
	Loss: 0.6300980448722839
	Loss: 0.6265809535980225
	Loss: 0.6313174962997437
	Loss: 0.6257287263870239
	Loss: 0.6196343898773193
	Loss: 0.6288171410560608
	Loss: 0.6256678700447083
	Loss: 0.6312776803970337
	Loss: 0.6298124194145203
	Loss: 0.623023271560669
	Loss: 0.6186664700508118
	Loss: 0.6294273138046265
	Loss: 0.6381123065948486
	Loss: 0.6196224689483643
	Loss: 0.6266673803329468
	Loss: 0.6224984526634216
	Loss: 0.6211059093475342
	Loss: 0.6231487989425659
	Loss: 0.6194751858711243
	Loss: 0.627687931060791
	Loss: 0.6079790592193604
	Loss: 0.631184

...

	Loss: 0.6338001489639282
	Loss: 0.6134812831878662
	Loss: 0.6199424266815186
	Loss: 0.6268001794815063
	Loss: 0.6164780259132385
	Loss: 0.6293928027153015
	Loss: 0.6441220641136169
	Loss: 0.6326923966407776
	Loss: 0.6299995183944702
	Loss: 0.6151749491691589
	Loss: 0.6283021569252014
	Loss: 0.6311413645744324
	Loss: 0.6297417283058167
	Loss: 0.6378257274627686
	Loss: 0.6078567504882812
	Loss: 0.6130244731903076
	Loss: 0.6269651651382446
	Loss: 0.629619836807251
	Loss: 0.6326199769973755
	Loss: 0.6312655210494995
	Loss: 0.6151825785636902
	Loss: 0.6237921714782715
	Loss: 0.62196284532547
	Loss: 0.6226214170455933
	Loss: 0.611522376537323
	Loss: 0.6305630207061768
	Loss: 0.6400603652000427
	Loss: 0.6193644404411316
	Loss: 0.6198063492774963
	Loss: 0.6154679656028748
	Loss: 0.6241692900657654
	Loss: 0.6209526658058167
	Loss: 0.6342540383338928
	Loss: 0.6206589937210083
	Loss: 0.6230875849723816
	Loss: 0.6369197368621826
	Loss: 0.6268793344497681
	Loss: 0.6262900829315186
	Loss: 0.6181927

	Loss: 0.6178494691848755
	Loss: 0.627988874912262
	Loss: 0.627121090888977
	Loss: 0.6248390674591064
	Loss: 0.6229868531227112
	Loss: 0.6195021271705627
	Loss: 0.6229713559150696
	Loss: 0.626674473285675
	Loss: 0.628991425037384
	Loss: 0.621582567691803
	Loss: 0.635023832321167
	Loss: 0.619195818901062
	Loss: 0.6057483553886414
	Loss: 0.6269432902336121
	Loss: 0.6309949159622192
	Loss: 0.6245425343513489
	Loss: 0.6306601166725159
	Loss: 0.6187434792518616
	Loss: 0.6337125897407532
	Loss: 0.6303881406784058
	Loss: 0.6199698448181152
	Loss: 0.6421549320220947
	Loss: 0.6329675316810608
	Loss: 0.6348466277122498
	Loss: 0.6284456849098206
	Loss: 0.6281005144119263
	Loss: 0.6150407791137695
	Loss: 0.6062561273574829
	Loss: 0.622149646282196
	Loss: 0.6261241436004639
	Loss: 0.6281264424324036
	Loss: 0.6392481327056885
	Loss: 0.6186788082122803
	Loss: 0.6310220956802368
	Loss: 0.6366627812385559
	Loss: 0.6207313537597656
	Loss: 0.6377288699150085
	Loss: 0.6268235445022583
	Loss: 0.61360830068

	Loss: 0.6275513768196106
	Loss: 0.6262758374214172
	Loss: 0.6175414323806763
	Loss: 0.6189855933189392
	Loss: 0.6230910420417786
	Loss: 0.6246307492256165
	Loss: 0.6131796836853027
	Loss: 0.6379362344741821
	Loss: 0.6204083561897278
	Loss: 0.6327682733535767
	Loss: 0.6266406774520874
	Loss: 0.6355498433113098
	Loss: 0.6217774748802185
	Loss: 0.6107760071754456
	Loss: 0.6146511435508728
	Loss: 0.6235070824623108
	Loss: 0.6377774477005005
	Loss: 0.6333087682723999
	Loss: 0.6234232187271118
	Loss: 0.6261414885520935
	Loss: 0.6223094463348389
	Loss: 0.6334912180900574
	Loss: 0.6369642615318298
	Loss: 0.6292092204093933
	Loss: 0.6197097897529602
	Loss: 0.6209163069725037
	Loss: 0.6135889887809753
	Loss: 0.6248356103897095
	Loss: 0.6250132322311401
	Loss: 0.6284612417221069
	Loss: 0.6219112277030945
	Loss: 0.6160970330238342
	Loss: 0.6197319030761719
	Loss: 0.6189688444137573
	Loss: 0.6257815957069397
	Loss: 0.6317495107650757
	Loss: 0.6403071880340576
	Loss: 0.6240238547325134
	Loss: 0.630

	Loss: 0.6289483904838562
	Loss: 0.6258925199508667
	Loss: 0.6310315132141113
	Loss: 0.6311903595924377
	Loss: 0.622822642326355
	Loss: 0.6244194507598877
	Loss: 0.6305981278419495
	Loss: 0.6288310885429382
	Loss: 0.6255046129226685
	Loss: 0.6249822974205017
	Loss: 0.620829701423645
	Loss: 0.6281284689903259
	Loss: 0.6285362839698792
	Loss: 0.6115854978561401
	Loss: 0.6263853311538696
	Loss: 0.6240026354789734
	Loss: 0.6226798892021179
	Loss: 0.6252806186676025
	Loss: 0.6343969702720642
	Loss: 0.6192352771759033
	Loss: 0.6252113580703735
	Loss: 0.6156273484230042
	Loss: 0.6332550644874573
	Loss: 0.6245337724685669
	Loss: 0.6199946999549866
	Loss: 0.614219069480896
	Loss: 0.6266838312149048
	Loss: 0.6392124891281128
	Loss: 0.6315894722938538
	Loss: 0.6298128366470337
	Loss: 0.6279421448707581
	Loss: 0.6267375349998474
	Loss: 0.6312183141708374
	Loss: 0.6317748427391052
	Loss: 0.6226462721824646
	Loss: 0.6301738023757935
	Loss: 0.6242436766624451
	Loss: 0.6300527453422546
	Loss: 0.629960

KeyboardInterrupt: 

In [52]:
label_id_mapping = trainset.label_map

id_label_mapping = dict()
for key in label_id_mapping.keys():
    id_label_mapping[label_id_mapping[key]] = key

def test_model(model, sentence, device = "cpu"):
    tokenized_sentence = torch.tensor([tokenizer.encode(sentence)])
    pos = torch.tensor([[0] * len(tokenized_sentence)])
    tags = torch.tensor([[1] * len(tokenized_sentence)])

    model = model.to(device)
    outputs = model(input_ids=tokenized_sentence.to(device), 
                    token_type_ids=pos.to(device), 
                    attention_mask=tags.to(device))

    logits = outputs[0]

    _, pred_labels = torch.max(logits, 2)

    out_labels = []
    for row in pred_labels:
        result = list(map(lambda x: id_label_mapping[int(x)], row))
        out_labels.append(result)
    #return tokenizer.tokenize(sentence), out_labels[0], logits
    return tokenizer.tokenize(sentence), out_labels[0][1:-1], logits[:, 1:-1]

In [18]:
sentence = "Dan will be deemed to have completed its delivery obligations before 2021-7-5 if in Niall's opinion, the Jeep Car satisfies the Acceptance Criteria, and Niall notifies Dan in writing that it is accepting the Jeep Car."

In [42]:
tokenized_sentence = torch.tensor([tokenizer.encode(sentence)])
pos = torch.tensor([[0] * len(tokenized_sentence)])
tags = torch.tensor([[1] * len(tokenized_sentence)])

model = model.to(device)
outputs = model(input_ids=tokenized_sentence.to(device), 
                token_type_ids=pos.to(device), 
                attention_mask=tags.to(device))

In [43]:
len(tokenizer.tokenize(sentence))

49

In [44]:
out = torch.sigmoid(outputs[0])

In [46]:
for i, text in enumerate(tokenizer.tokenize(sentence)):
    print(f"{text}: {out.view(-1)[i+1]:.2f}")

Dan: 0.91
ĠWill: 0.09
Ġbe: 0.00
Ġdeemed: 0.00
Ġto: 0.00
Ġhave: 0.00
Ġcompleted: 0.00
Ġits: 0.01
Ġdelivery: 0.00
Ġobligations: 0.00
Ġbefore: 0.00
Ġ2021: 0.02
-: 0.00
7: 0.00
-: 0.00
5: 0.00
Ġif: 0.00
Ġin: 0.00
ĠNi: 0.87
all: 0.19
's: 0.01
Ġopinion: 0.00
,: 0.00
Ġthe: 0.00
ĠJeep: 0.67
ĠCar: 0.24
Ġsatisfies: 0.00
Ġthe: 0.01
ĠAccept: 0.21
ance: 0.05
ĠCrit: 0.53
eria: 0.23
,: 0.00
Ġand: 0.00
ĠNi: 0.88
all: 0.05
Ġnot: 0.00
ifies: 0.00
ĠDan: 0.90
Ġin: 0.00
Ġwriting: 0.00
Ġthat: 0.00
Ġit: 0.02
Ġis: 0.00
Ġaccepting: 0.00
Ġthe: 0.00
ĠJeep: 0.73
ĠCar: 0.24
.: 0.00


In [21]:
sentence = "Dan Will be deemed to have completed its delivery obligations before 2021-7-5 if in Niall's opinion, the Jeep Car satisfies the Acceptance Criteria, and Niall notifies Dan in writing that it is accepting the Jeep Car."
sen, pred, logits = test_model(model, sentence, device = 'cpu')

NameError: name 'test_model' is not defined

In [ ]:
a = tokenizer.tokenize(sentence)[1]

In [ ]:
np.array(sen)

In [ ]:
np.array(pred)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def interact_word(i):
    print(i)
    print(sen[i])
    target = out[i]

    for i in range(len(target)):
        print(f"{i} {id_label_mapping[i].ljust(6)} \t: {target[i]:.5f}")

In [ ]:
out = logits[0]
interact(lambda x: interact_word(x), x=widgets.IntSlider(min=0, max=len(sen)-1, step=1, value=0))

In [ ]:
print("OK")